In [1]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv, find_dotenv
import pinecone
from sentence_transformers import SentenceTransformer

In [2]:
files = pd.read_csv("course_descriptions.csv", encoding = "ANSI")

In [3]:
def create_course_description(row):
    return f'''The course name is {row["course_name"]}, the slug is {row["course_slug"]},
            the technology is {row["course_technology"]} and the course topic is {row["course_topic"]}'''

In [4]:
pd.set_option('display.max_rows', 106)
files['course_description_new'] = files.apply(create_course_description, axis = 1)
print(files["course_description_new"])

0      The course name is Introduction to Tableau, th...
1      The course name is The Complete Data Visualiza...
2      The course name is Introduction to R Programmi...
3      The course name is Data Preprocessing with Num...
4      The course name is Introduction to Data and Da...
5      The course name is Data Cleaning and Preproces...
6      The course name is Introduction to Business An...
7      The course name is Data Analysis with Excel Pi...
8      The course name is SQL, the slug is sql,\n    ...
9      The course name is Credit Risk Modeling in Pyt...
10     The course name is Python Programmer Bootcamp,...
11     The course name is SQL + Tableau + Python, the...
12     The course name is Introduction to Jupyter, th...
13     The course name is Statistics, the slug is sta...
14     The course name is Mathematics, the slug is ma...
15     The course name is Introduction to Excel, the ...
16     The course name is Probability, the slug is pr...
17     The course name is Start

In [5]:
%load_ext dotenv
%dotenv

In [6]:
load_dotenv(find_dotenv(), override = True)

True

In [7]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [8]:
index_name = "my-index"
dimension = 384
metric = "cosine"

In [9]:
if index_name in [index.name for index in pc.list_indexes()]:
    pc.delete_index(index_name)
    print(f"{index_name} succesfully deleted.")
else:
     print(f"{index_name} not in index list.")

my-index succesfully deleted.


In [10]:
pc.create_index(
    name = index_name, 
    dimension = dimension, 
    metric = metric, 
    spec = ServerlessSpec(
        cloud = "aws", 
        region = "us-east-1")
    )

{
    "name": "my-index",
    "metric": "cosine",
    "host": "my-index-67t99i9.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:
index = pc.Index(index_name)

## Embedding the data

In [24]:
model = SentenceTransformer("all-MiniLM-L6-v2")
#model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [25]:
def create_embeddings(row):
    combined_text = ' '.join([str(row[field]) for field in ['course_description', 'course_description_new', 'course_description_short']])
    embedding = model.encode(combined_text, show_progress_bar = False)
    return embedding

In [26]:
files["embedding"] = files.apply(create_embeddings, axis = 1)

C:\Users\Marcus\anaconda3\envs\langchain_env_py312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [27]:
vectors_to_upsert = [(str(row["course_name"]), row["embedding"].tolist()) for _, row in files.iterrows()]
index.upsert(vectors = vectors_to_upsert)

print("Data upserted to Pinecone index")

Data upserted to Pinecone index


## Semantic search

In [16]:
query = "clustering"
query_embedding = model.encode(query, show_progress_bar = False).tolist()

C:\Users\Marcus\anaconda3\envs\langchain_env_py312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [17]:
query_results = index.query(
    vector = [query_embedding],
    top_k = 12, 
    include_values = True
)

In [18]:
query_results

{'matches': [{'id': 'Machine Learning in Excel',
              'score': 0.354685217,
              'values': [-0.0183002222,
                         -0.0279485676,
                         -0.025320353,
                         -0.0126938475,
                         -0.024036685,
                         -0.0219840892,
                         -0.0511236638,
                         -0.053580042,
                         0.00997647643,
                         0.028228702,
                         -0.0408324562,
                         -0.036268644,
                         0.0683277473,
                         -0.0348471,
                         -0.00728513766,
                         0.0366663076,
                         -0.00331020402,
                         -0.00411818968,
                         -4.75332308e-05,
                         -0.0627969131,
                         0.0846960098,
                         0.0300104581,
                         -0.0528304949,
   

In [19]:
score_threshold = 0.3
for match in query_results["matches"]:
    if match['score'] >= score_threshold:
        print(f"Matched item ID: {match['id']}, score: {match['score']}")

Matched item ID: Machine Learning in Excel, score: 0.354685217
Matched item ID: Machine Learning with K-Nearest Neighbors, score: 0.313649178
